### __Implement a hyperparameter optimization strategy (grid search) for a neural network model.__ 

In [6]:
import numpy as np
import keras_tuner as kt
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Input
from tensorflow.keras.optimizers import SGD
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import seaborn as sns
import tensorflow as tf

In [7]:
df = sns.load_dataset('iris')
df.head()

x = df.drop(columns=['sepal_length'])
y = df['sepal_length']

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

le = LabelEncoder()
x_train['species'] = le.fit_transform(x_train['species'])
x_test['species']= le.transform(x_test['species'])

In [9]:
def test_model(hp):
    model=Sequential()
    model.add(Input(shape=(x_train.shape[1],)))
    model.add(Dense(hp.Int('unit1',min_value=32,max_value=128,step=32),activation=hp.Choice('activation1',['relu','tanh','sigmoid'])))
    model.add(Dense(hp.Int('unit2',min_value=32,max_value=128,step=32),activation=hp.Choice('activation2',['relu','tanh','sigmoid'])))
    model.add(Dense(1))

    model.compile(optimizer=SGD(hp.Choice('optimize',[0.00001,0.0001,0.001]),momentum=hp.Choice('momentum',[0.0,0.5,0.9])),loss='mean_squared_error',metrics=['mae'])

    return model
    

In [10]:
tuner = kt.GridSearch(
    test_model,
    objective='val_mae',  
    max_trials=10,
    executions_per_trial=1, 
    directory='test', 
    project_name='iris_reg2' 
)

In [11]:
tuner.search(x_train, y_train, epochs=10, validation_data=(x_test, y_test))
best_hp = tuner.get_best_hyperparameters(num_trials=1)[0]
best_model = tuner.hypermodel.build(best_hp)
best_model.fit(x_train, y_train, epochs=10, validation_data=(x_test, y_test))
loss, mae = best_model.evaluate(x_test, y_test)
print(f"Test MAE: {mae}")

Trial 10 Complete [00h 00m 04s]
val_mae: 6.4243011474609375

Best val_mae So Far: 0.35166576504707336
Total elapsed time: 00h 00m 42s
Epoch 1/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 99ms/step - loss: 31.3742 - mae: 5.4678 - val_loss: 7.2666 - val_mae: 2.6802
Epoch 2/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step - loss: 4.4621 - mae: 1.9042 - val_loss: 9.9625 - val_mae: 2.9042
Epoch 3/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step - loss: 6.6890 - mae: 2.3028 - val_loss: 0.9676 - val_mae: 0.9494
Epoch 4/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step - loss: 2.3243 - mae: 1.3798 - val_loss: 5.0933 - val_mae: 2.2096
Epoch 5/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step - loss: 3.5358 - mae: 1.7468 - val_loss: 0.1069 - val_mae: 0.2750
Epoch 6/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step - loss: 0.9595 - mae: 0.8098 - val_loss: 1.3452 - val_mae: 1.0892
Epoch 7/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step - loss: 0.9223 - mae: 0.8278 - val_loss: 0.7835 - val_mae: 0.8287
Epoch 8/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step - loss: 0.8